In [ ]:
# COGS109
# Homework 4
# Gustav Sto. Tomas
# A15358078

import pandas as pd
import pandas.tseries
from pandas.core import datetools
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

df = pd.read_csv('hw3_divseq_data.csv')

In [ ]:
df.head()

In [ ]:
#2a
logreg2 = smf.logit(formula = 'mature ~ Lars2 + Malat1', data=df).fit()
print(logreg2.summary())
print(logreg2.pvalues)
pred_both = df[['Lars2','Malat1']]

df['pred_both'] = logreg2.predict(pred_both)
df['pred_both_mature'] = 1*(df.pred_both > 0.5)
df['pred_both_immature'] = 1 *(df.pred_both < 0.5)

In [6]:
# 2a : confusion matrix

conf_mat = confusion_matrix(df.mature, df.pred_both_mature,)
print(conf_mat)

ok = np.zeros((3,3))
#print(ok)
ok[0,2] = 1
ok[2,0] = 1
ok[1,1] = conf_mat[0,0]
ok[1,2] = conf_mat[0,1]
ok[2,1] = conf_mat[1,0]
ok[2,2] = conf_mat[1,1]
print(ok)

con = pd.DataFrame(data=conf_mat, index=['true_immature','true_mature'],columns=['pred_immature','pred_mature'])
print(con)


correct_m = conf_mat[1,1]
correct_im = conf_mat[0,0]
correct = conf_mat[0,0]+conf_mat[1,1]
print()
print()
print('correctly predicted mature (TP):', correct_m)
print('correctly predicted immature (TN):', correct_im)
print('correctly predicted:', correct)
print('correctly predicted:', correct/len(df))

[[618  34]
 [ 45 120]]
[[   0.    0.    1.]
 [   0.  618.   34.]
 [   1.   45.  120.]]
               pred_immature  pred_mature
true_immature            618           34
true_mature               45          120


correctly predicted mature (TP): 120
correctly predicted immature (TN): 618
correctly predicted: 738
correctly predicted: 0.903304773562


In [9]:
#2 b and c

df['pred_both_20'] = 1*(df.pred_both > 0.2)
df['pred_both_80'] = 1*(df.pred_both > 0.8)



conf_mat2 = confusion_matrix(df.mature, df.pred_both_20,)
#print(conf_mat2)

conf_mat8 = confusion_matrix(df.mature, df.pred_both_80,)
#print(conf_mat8)

con2 = pd.DataFrame(data=conf_mat2, index=['true_immature','true_mature'],columns=['pred_immature','pred_mature'])
print('confusion matrix with threshold 0.2:')
print(con2)
print()



specificity2 = conf_mat2[0][0]/(conf_mat2[0][0]+conf_mat2[0][1])
sensitivity2 = conf_mat2[1][1]/(conf_mat2[1][0]+conf_mat2[1][1])

#sensitivity2 = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
print("Sensitivity 0.2: " + str(sensitivity2))
#specificity2 = conf_mat2[1,1]/(conf_mat2[1,0]+conf_mat2[1,1])
print("Specificity 0.2: " + str(specificity2))
print('with a threshold of 0.2, specificity is somewhat lower, but sensitivity is very high, meaning fewer false positives')

print()

con8 = pd.DataFrame(data=conf_mat8, index=['true_immature','true_mature'],columns=['pred_immature','pred_mature'])
print('confusion_matrix with threshold 0.8:')
print(con8)
print()


specificity8 = conf_mat8[0][0]/(conf_mat8[0][0]+conf_mat8[0][1])
sensitivity8 = conf_mat8[1][1]/(conf_mat8[1][0]+conf_mat8[1][1])

#sensitivity8 = conf_mat8[0,0]/(conf_mat8[0,0]+conf_mat8[0,1])
print("Sensitivity 0.8: " + str(sensitivity8))
#specificity8 = conf_mat8[1,1]/(conf_mat8[1,0]+conf_mat8[1,1])
print("Specificity 0.8: " + str(specificity8))
print('with a threshold of 0.8, specificity is somewhat higher but sensitivity is much lower, meaning more false negatives.')
print()
print('more actually mature neurons are correctly classified as mature with a threshold of 0.2. a threshold of 0.8 instead classifies even more of the immature neurons correctly than the bayesian (0.5) classifier died, but many more mature neurons are falsely classified as immature (false negatives). since we want to make better classifications for mature neurons, a threshold of 0.2 is to prefer.')


confusion matrix with threshold 0.2:
               pred_immature  pred_mature
true_immature            575           77
true_mature                7          158

Sensitivity 0.2: 0.957575757576
Specificity 0.2: 0.881901840491
with a threshold of 0.2, specificity is somewhat lower, but sensitivity is very high, meaning fewer false positives

confusion_matrix with threshold 0.8:
               pred_immature  pred_mature
true_immature            643            9
true_mature               90           75

Sensitivity 0.8: 0.454545454545
Specificity 0.8: 0.986196319018
with a threshold of 0.8, specificity is somewhat higher but sensitivity is much lower, meaning more false negatives.

more actually mature neurons are correctly classified as mature with a threshold of 0.2. a threshold of 0.8 instead classifies even more of the immature neurons correctly than the bayesian (0.5) classifier died, but many more mature neurons are falsely classified as immature (false negatives). since we want 

In [ ]:
### some test stuff

falseP = []
threshold = np.arange(0,1,0.1)
for i in range(len(threshold)):
    cat_labels = 1*(df.pred_both > threshold[i])
    cons = confusion_matrix(df.mature,cat_labels)
    specificity = cons[0][0]/(cons[0][0]+cons[0][1])
    sensitivity = cons[1][1]/(cons[1][0]+cons[1][1])
    #sensitivity = cons[0,0]/(cons[0,0]+cons[0,1])
    #specificity = cons[1,1]/(cons[1,0]+cons[1,1])
    fpr = 1-specificity
    tpr = sensitivity
    falseP.append(fpr)
    #print(fpr)
    print(cons)

print(falseP)
    

trueP = []
threshold = np.arange(0,1,0.1)
for i in range(len(threshold)):
    cat_labels = 1*(df.pred_both < threshold[i])
    cons = confusion_matrix(df.mature,cat_labels)
    specificity = cons[0][0]/(cons[0][0]+cons[0][1])
    sensitivity = cons[1][1]/(cons[1][0]+cons[1][1])
    #sensitivity = cons[0,0]/(cons[0,0]+cons[0,1])
    #specificity = cons[1,1]/(cons[1,0]+cons[1,1])
    fpr = 1-specificity
    tpr = sensitivity
    trueP.append(tpr)
    #print(tpr)
    print(cons)

print(trueP)    
    

fig,ax = plt.subplots()
ax.set_autoscale_on(False)
plt.plot(threshold,falseP)
plt.xlabel('threshold')
plt.ylabel('false positive rate')
plt.show()


f = lambda x: falseP

fig,ax = plt.subplots()
ax.set_autoscale_on(False)
plt.plot(threshold,trueP)
plt.xlabel('threshold')
plt.ylabel('true positive rate')
plt.show()



fig,ax = plt.subplots()
ax.set_autoscale_on(False)
plt.plot(falseP,trueP)
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()




conf = confusion_matrix(df.mature,cat_labels)
print(conf)
print(len(df.mature)-conf[0,1])
sensitivity = conf[1][1]/(conf[1][0]+conf[1][1])
print("Sensitivity:", sensitivity)
specificity = conf[0][0]/(conf[0][0]+conf[0][1])
print("Specificity:", specificity)

In [ ]:
#2 d, e, and f

y = df.mature
scores = df.pred_both
fpr, tpr, thresholds = roc_curve(y, scores)#, cat_labels)

fig, ax = plt.subplots()
plt.plot(thresholds,fpr)
plt.xlabel('threshold')
plt.ylabel('false positive rate')
plt.show()


fig,ax = plt.subplots()
plt.plot(thresholds,tpr)
plt.xlabel('threshold')
plt.ylabel('true positive rate')
plt.show()


fig,ax = plt.subplots()
#plt.plot(roc_curve(y,scores))
plt.plot(fpr,tpr,'r.')
#ax.scatter(df.pred_both,roc_curve(df.mature,df.pred_both))
#plt.plot(thresholds==0.5, 'b-')
#for i in range(len(thresholds)):
#    if thresholds[i] > 0.5 < 0.51:
#        plt.plot(fpr, tpr, marker='o', markersize=3, color="blue")
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()



"""fig,ax = plt.subplots()
#plt.plot(roc_curve(y,scores))
plt.plot(fpr,tpr,'r.')
#ax.scatter(df.pred_both,roc_curve(df.mature,df.pred_both))
#plt.plot(thresholds==0.5, 'b-')
#for i in range(len(thresholds)):
    cats = 1*(scores > thresholds[i])
    #if thresholds[i] > 0.5 < 0.51:
    #plt.plot(cats,'b.')#, marker='o', markersize=0.01, color="blue")
    #for i in cats:
    #    if scores == 0.5
    #        plt.plot(fpr,tpr,'b.')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()"""

In [ ]:
p = lambda n: (1-(1-1/n)^n)
x = np.arange(1,100000)
fig,ax = plt.subplots()
plt.plot(x,p(x))
plt.show()

#pr = function(n) return(1 - (1 - 1/n)^n)
#x = 1:1e+05
#plot(x, pr(x))